In [39]:
import numpy as np
import pandas as pd

In [40]:
def rolling_forecast_origin(train,min_train_size,horizon):
  #rolling forecast origin generator
  for i in range(len(train)-min_train_size-horizon+1):
    split_train=train[:min_train_size+i]
    split_val=train[min_train_size+i:min_train_size+i+horizon]
    yield split_train, split_val

In [41]:
full_series=[2502,2414,2800,2143,2708,1900,2333,2222,1234,3456]

test=full_series[-2:]
train=full_series[:-2]
print("full training set: {0}".format(train)) #this 0 tells index of data in format
print("hidden test set: {0}".format(test))

full training set: [2502, 2414, 2800, 2143, 2708, 1900, 2333, 2222]
hidden test set: [1234, 3456]


In [42]:
cv_rolling=rolling_forecast_origin(train,min_train_size=4,horizon=1)
cv_rolling

<generator object rolling_forecast_origin at 0x7cdaf2d32180>

In [43]:
i=0
for cv_train,cv_val in cv_rolling:
  print(f"CV[{i+1}]")
  print(f"Train:\t{cv_train}")
  print(f"Val:\t{cv_val}")
  print("-----------------")
  i+=1

#we are predicting the future every time as it is time series data so
#every time predicting future and next time that future data becomes part
#of data and we predict next future data point considering

CV[1]
Train:	[2502, 2414, 2800, 2143]
Val:	[2708]
-----------------
CV[2]
Train:	[2502, 2414, 2800, 2143, 2708]
Val:	[1900]
-----------------
CV[3]
Train:	[2502, 2414, 2800, 2143, 2708, 1900]
Val:	[2333]
-----------------
CV[4]
Train:	[2502, 2414, 2800, 2143, 2708, 1900, 2333]
Val:	[2222]
-----------------


In [44]:
#for two steps in the future
cv_rolling=rolling_forecast_origin(train,min_train_size=4,horizon=2)
i=0
for cv_train,cv_val in cv_rolling:
  print(f"CV[{i+1}]")
  print(f"Train:\t{cv_train}")
  print(f"Val:\t{cv_val}")
  print("-----------------")
  i+=1

CV[1]
Train:	[2502, 2414, 2800, 2143]
Val:	[2708, 1900]
-----------------
CV[2]
Train:	[2502, 2414, 2800, 2143, 2708]
Val:	[1900, 2333]
-----------------
CV[3]
Train:	[2502, 2414, 2800, 2143, 2708, 1900]
Val:	[2333, 2222]
-----------------


In [45]:
#sliding window approach
#in this there is a fix window size like when we add next future
#prediction data to our
#train data then we also keep on removing older data from train set

In [46]:
def sliding_window(train,window_size,horizon):
  #sliding window generator
  for i in range(len(train)-window_size-horizon+1):
    split_train=train[i:window_size+i]
    split_val=train[window_size+i:window_size+i+horizon]
    yield split_train, split_val

In [47]:
cv_sliding=sliding_window(train,window_size=4,horizon=1)
i=0
for cv_train,cv_val in cv_sliding:
  print(f"CV[{i+1}]")
  print(f"Train:\t{cv_train}")
  print(f"Val:\t{cv_val}")
  print("-----------------")
  i+=1

CV[1]
Train:	[2502, 2414, 2800, 2143]
Val:	[2708]
-----------------
CV[2]
Train:	[2414, 2800, 2143, 2708]
Val:	[1900]
-----------------
CV[3]
Train:	[2800, 2143, 2708, 1900]
Val:	[2333]
-----------------
CV[4]
Train:	[2143, 2708, 1900, 2333]
Val:	[2222]
-----------------


In [48]:
#cross validation example

In [49]:
def cross_validation_score(model, train, cv, metric):
  #we pass model for training, train dataset, crossvalidation generator
  #and metric for evaluation
  cv_scores=[]
  for cv_train, cv_test in cv:
    model.fit(cv_train)
    preds=model.predict(horizon=len(cv_test))
    print(f"preds: {preds}")
    score=metric(y_true=cv_test,y_pred=preds)
    cv_scores.append(score)
  return np.array(cv_scores)

In [50]:
#!pip install forecast
from forecast.baseline import SNaive, Naive1 #SNaive is sesonal naive
from sklearn.metrics import mean_absolute_error

#learn about time series data analysis then do dive into this
#because predictive time series data is actually different then
#then simply applying ML models

#i have copied the code but library is not working and also not
#understood with clearity

ModuleNotFoundError: ignored

In [ ]:
#load the ED attendance dataset
ed_month = pd.read_csv('data/ed_mth_ts.csv', index_col='date', parse_dates=True)
ed month.index.freq= 'MS'
arrival_rate = ed_month ['arrivals ] / ed_month.index.days_in_month
arrival_rate.shape

In [ ]:
#Train test split

SPLIT DATE = '2016-06-01'
train = arrival_ rate.loc[arrival_rate.index < SPLIT_DATE]
test = arrival_rate.loc[arrival_rate.index >= SPLIT_DATE]
train.shape

In [ ]:
cv_sliding = sliding_window(train, window_size=24, horizon=12)
cv_scores_1 = cross_validation_score(model=SNaive(period=12),
                                     train=train,
                                     cv=cv_sliding,
                                     metric=mean_absolute_error)
pd. DataFrame (cv_scores_1).describe()

In [ ]:
cv_sliding = sliding_window(train, window_size=24, horizon=12)
cv_scores_2 = cross_validation_score (model=Naive1(),
                                      train=train,
                                      cv=cv_sliding,
                                      metric=mean_absolute_error)
pd.DataFrame (cv_scores_2).describe ()

In [ ]:
ax = pd.DataFrame (cv_scores_1).plot( figsize= (12,4))
pd.Dataframe(cv_scores_ 2).plot(ax=ax)
ax.legend ( ['SNaive', 'Naivel1'])

In [ ]:
model = SNaive(period=12)
model.fit(train)
preds = model.predict(horizon=12)

In [ ]:
mean_absolute. _error(test, preds)